In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from datasets import load_dataset, concatenate_datasets, Dataset
import copy
from tqdm import tqdm  # For tracking training progress


In [2]:
# Example list of labels
label_names = ['cat', 'dog', 'bird', 'fish', 'car', 'aircraft', 'flower', 'truck', 'parachute', 'mushroom']

# Create a mapping from label names to indices
label_to_index = {label: idx for idx, label in enumerate(label_names)}


In [3]:
import torch
import numpy as np
import random

seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

In [4]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3), # Convert grayscale to 3 channels (RGB)
    transforms.Resize((256, 256)),  # Resize all images to 256x256
    transforms.ToTensor(),          # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

from PIL import Image

def apply_transform(example):
    # Check if 'example['image']' is a list (batch of images)
    transformed_images = [transform(img) for img in example['image']]
    labels = [label_to_index[label] for label in example['label']]
    # Return the transformed images and the unchanged labels
    return {
        'image': transformed_images,  # Stack to create a single tensor
        'label': torch.tensor(labels)  # Convert labels to tensor
    }

# Apply the transformations to the dataset (train + test split for each 


In [5]:
def prepare_custom_dataloader(dataset, batch_size=16):
    # Apply the transformation to each sample in the dataset
    dataset = dataset.with_transform(apply_transform)
    
    # Create dataloaders
    train_loader = DataLoader(dataset['train'], batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(dataset['test'], batch_size=batch_size, shuffle=False)
    return train_loader, test_loader

In [6]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        # Define layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(32 * 64 * 64, 128)  # Adjust based on output size from conv layers
        self.fc2 = nn.Linear(128, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(nn.ReLU()(self.conv1(x)))  # Conv Layer 1
        x = self.pool((nn.ReLU()(self.conv2(x))))  # Conv Layer 2
        x = x.view(-1, 32 * 64 * 64)  # Flatten for fully connected layer
        x = nn.ReLU()(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x) 
        return x

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


# Define Label Smoothing function
def label_smoothing_loss(logits, target, smoothing=0.2):
    num_classes = logits.size(1)
    one_hot_target = F.one_hot(target, num_classes=num_classes).float()
    smooth_target = one_hot_target * (1 - smoothing) + smoothing / num_classes
    log_probs = F.log_softmax(logits, dim=-1)
    
    # Calculate the loss
    return -torch.sum(smooth_target * log_probs, dim=-1).mean()

def label_smoothing_huber_loss(logits, target, smoothing=0.2):
    num_classes = logits.size(1)
    
    # One-hot encode the target
    one_hot_target = F.one_hot(target, num_classes=num_classes).float()

    # Apply label smoothing
    smooth_target = one_hot_target * (1 - smoothing) + smoothing / num_classes

    # Apply the Huber loss (Smooth L1 Loss)
    huber_loss = F.smooth_l1_loss(logits, smooth_target, reduction='mean')
    
    return huber_loss
   

def train_local_model_huber(model, train_loader, epochs, smoothing=0.2):
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
    model.train()

    for epoch in range(epochs):
        for batch in train_loader:
            images = batch['image']
            labels = batch['label']
            
            optimizer.zero_grad()
            outputs = model(images)  # Get model predictions
            
            # Use the label smoothing function with Huber loss
            loss = label_smoothing_huber_loss(outputs, labels, smoothing=smoothing)
            loss.backward()
            optimizer.step()

    return model.state_dict()

In [18]:
# Function to update local model on each client
def train_local_model(model, train_loader, epochs, smoothing=0.1):
    #criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    criterion = nn.L1Loss()
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
    model.train()
    
    for epoch in range(epochs):
        for batch in train_loader:
            images = batch['image']  # This should be a tensor
            labels = batch['label'] 
            # Convert labels to one-hot encoding
            labels_one_hot = torch.nn.functional.one_hot(labels, num_classes=10).float()

            # Apply label smoothing
            labels_smooth = labels_one_hot * (1 - smoothing) + smoothing /10

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels_smooth)
            loss.backward()
            optimizer.step()
    return model.state_dict()  # Return the trained model's weights

# Function to average weights across clients
def federated_avg(global_model, client_weights):
    avg_weights = copy.deepcopy(client_weights[0])
    
    for key in avg_weights.keys():
        for i in range(1, len(client_weights)):
            avg_weights[key] += client_weights[i][key]
        avg_weights[key] = avg_weights[key] / len(client_weights)
    
    global_model.load_state_dict(avg_weights)
    return global_model

def weighted_federated_averaging(global_model, client_weights, client_data_sizes, client_noise_fractions):
    # Compute the weights for each client
    total_weight = sum([size * (1 - noise) for size, noise in zip(client_data_sizes, client_noise_fractions)])
    normalized_weights = [(size * (1 - noise)) / total_weight for size, noise in zip(client_data_sizes, client_noise_fractions)]
    
    # Initialize global model parameters with zeros
    global_params = {k: torch.zeros_like(v) for k, v in client_weights[0].items()}
    
    # Perform weighted aggregation
    for client_idx, client_weight_dict in enumerate(client_weights):
        client_weight = normalized_weights[client_idx]
        for param_key in global_params.keys():
            global_params[param_key] += client_weight_dict[param_key] * client_weight  # Weighted sum of client models
    
    # Load the updated global weights into the global model
    global_model.load_state_dict(global_params)
    return global_model


In [9]:
dataset_1 = load_dataset("AnnantJain/client1_federated_dataset_modified")
dataset_2 = load_dataset("AnnantJain/client2_federated_dataset_modified")
dataset_3 = load_dataset("AnnantJain/client3_federated_dataset_modified")
dataset_4 = load_dataset("AnnantJain/client4_federated_dataset_modified")
dataset_5 = load_dataset("AnnantJain/client5_federated_dataset_modified")

In [10]:
train_loader_1, test_loader_1 = prepare_custom_dataloader(dataset_1)
train_loader_2, test_loader_2 = prepare_custom_dataloader(dataset_2)
train_loader_3, test_loader_3 = prepare_custom_dataloader(dataset_3)
train_loader_4, test_loader_4 = prepare_custom_dataloader(dataset_4)
train_loader_5, test_loader_5 = prepare_custom_dataloader(dataset_5)

In [11]:
for batch in train_loader_1:
    # Since DataLoader returns batches, you may want to just take the first batch
    images = batch['image']  # This should be a tensor
    labels = batch['label'] 
    break  # Exit after the first batch

# Now 'images' and 'labels' are tensors
print("Sample Image Tensor Shape:", images.shape)  # Print the shape of the image tensor
print("Sample Label Tensor:", labels)  # Print the label tensor

Sample Image Tensor Shape: torch.Size([16, 3, 256, 256])
Sample Label Tensor: tensor([2, 2, 8, 5, 5, 8, 6, 6, 6, 5, 2, 8, 2, 5, 6, 5])


In [12]:
# Initialize global model (shared across clients)
num_classes = 10  # Adjust based on your specific case
global_model = SimpleCNN(num_classes=num_classes)


In [13]:
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in test_loader:
            images = batch['image']  # This should be a tensor
            labels = batch['label'] 
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

In [ ]:
client_data_sizes = [1800, 3000, 2400, 2000, 3200]  # Number of samples per client
client_noise_fractions = [0.72, 0.63, 0.66, 0.6, 0.58]

In [20]:
# Number of communication rounds
num_rounds = 5

# Number of local epochs for each client
local_epochs = 2

# Perform Federated Averaging
for round_num in range(num_rounds):
    print(f"Round {round_num + 1}/{num_rounds}")
    
    # Collect weights from all clients
    client_weights = []
    
    # Simulate client training
    for client_id, train_loader in enumerate([train_loader_1, train_loader_2, train_loader_3, train_loader_4, train_loader_5]):
        local_model = copy.deepcopy(global_model)  # Each client starts from the global model
        local_weights = train_local_model_huber(local_model, train_loader, epochs=local_epochs)  # Train locally
        client_weights.append(local_weights)  # Store client weights
    
    # Federated averaging (aggregation) step
    global_model = federated_avg(global_model, client_weights)
    
    print(f"Completed round {round_num + 1}")

    # Evaluate the global model on the test data from each client
    for client_id, test_loader in enumerate([test_loader_1, test_loader_2, test_loader_3, test_loader_4, test_loader_5]):
        accuracy = evaluate_model(global_model, test_loader)
        print(f"Client {client_id + 1} Test Accuracy: {accuracy:.2f}%")


Round 1/5
Completed round 1
Client 1 Test Accuracy: 0.37%
Client 2 Test Accuracy: 11.78%
Client 3 Test Accuracy: 0.28%
Client 4 Test Accuracy: 18.33%
Client 5 Test Accuracy: 11.04%
Round 2/5


KeyboardInterrupt: 